In [ ]:
# signal_analyzer.py
import pandas as pd
from datetime import datetime, timedelta

# Load full history
df = pd.read_csv("data/nifty_options_chain.csv")
df['date'] = pd.to_datetime(df['date'])

# Filter last 5 days
cutoff = datetime.today() - timedelta(days=5)
df_5d = df[df['date'] > cutoff]

# Group by strike & calculate OI trend
summary = (
    df_5d.groupby('strikePrice')
    .agg(avg_OI=('OI', 'mean'),
         OI_change=('OI', lambda x: x.iloc[-1] - x.iloc[0]),
         LTP_change=('LTP', lambda x: x.iloc[-1] - x.iloc[0]),
         avg_volume=('volume', 'mean'))
    .reset_index()
)

# Flag potential smart money activity
signals = summary[
    (summary['OI_change'] > 10000) &
    (summary['avg_volume'] < 1000) &
    (summary['LTP_change'] > 5)
]

print("📊 Potential Smart Call Buildup Setups:")
print(signals.sort_values("OI_change", ascending=False))
